<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2021</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Разбор задач: Classes</b><br/>
    <br/>
    <font>Никита Бондарцев</font><br/>
</center>

### 1. orders  → etalon (1)

In [1]:
from dataclasses import dataclass, field, InitVar
from abc import abstractmethod, ABC
from typing import Union


DISCOUNT_PERCENTS = 15


@dataclass(order=True, frozen=True)
class Item:
    item_id: int = field(compare=False)
    title: str
    cost: int

    def __post_init__(self) -> None:
        assert self.title
        assert self.cost > 0

        
# Do not remove `# type: ignore`
# It is [a really old issue](https://github.com/python/mypy/issues/5374)
@dataclass  # type: ignore
class Position(ABC):
    item: Item

    @property
    @abstractmethod
    def cost(self) -> Union[float, int]:
        pass


In [2]:
@dataclass
class CountedPosition(Position):
    count: int = 1

    @property
    def cost(self) -> Union[float, int]:
        return self.count * self.item.cost


@dataclass
class WeightedPosition(Position):
    weight: float = 1.0

    @property
    def cost(self) -> Union[float, int]:
        return self.weight * self.item.cost

@dataclass
class Order:
    order_id: int
    positions: list[Position] = field(default_factory=list)
    cost: int = field(init=False)
    have_promo: InitVar[bool] = False

    def __post_init__(self, have_promo: bool) -> None:
        cost = sum(position.cost for position in self.positions)
        if have_promo:
            promo_multiplier = (1.0 - DISCOUNT_PERCENTS / 100)
            cost *= promo_multiplier
        self.cost = int(cost)


### 1. orders  → злоупотребление # type:ignore (2)

In [14]:
 
# Do not remove `# type: ignore`
# It is [a really old issue](https://github.com/python/mypy/issues/5374)
@dataclass  # type: ignore
class Position(ABC):
    item: Item

    @abstractmethod
    def cost(self):  # type:ignore
        pass

    def __post_init__(self):  # type:ignore
        self.cost = self.cost()

    def __getattr__(self, item):  # type:ignore
        if item == 'cost':
            return self.cost

        
@dataclass
class Order:
    order_id: int
    positions: list[Position] = field(default_factory=list)
    cost: int = 0
    have_promo: InitVar[bool] = False

    def __post_init__(self, have_promo):  # type:ignore
        for elem in self.positions:
            self.cost += elem.cost
        self.cost = int(self.cost * (100 - DISCOUNT_PERCENTS) / 100) if have_promo else int(self.cost)


### 1. orders  → ручная реализация `__lt__` (3)

In [ ]:
@dataclass(frozen=True)
class Item:
    item_id: int
    title: str
    cost: int

    def __post_init__(self) -> None:
        assert self.title != '' and self.cost > 0

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Item):
            return NotImplemented
        # Предметы должны сравниваться по названию, а после по цене
        if self.title == other.title:
            return True
        elif self.cost == other.cost:
            return True
        else:
            return False

    def __lt__(self, other: object) -> bool:
        if not isinstance(other, Item):
            return NotImplemented
        if self.title < other.title:
            return True
        elif self.title > other.title:
            return False
        else:
            if self.cost < other.cost:
                return True
            elif self.cost > other.cost:
                return False
            else:
                return False

### 1. orders  → игнорирование константы количества процентов (4)

In [ ]:
@dataclass
class Order:
    order_id: int
    cost: int = 0
    positions: list[Position] = field(default_factory=list)
    have_promo: InitVar[bool] = field(init=True, default=False)

    def __post_init__(self, have_promo: bool) -> None:
        sum_ = 0
        for position in self.positions:
            sum_ += position.cost  # type: ignore
        if have_promo:
            self.cost = int(sum_ * 0.85)
        else:
            self.cost = int(sum_)


### 1. orders  → значение по умолчанию через None (5)

In [ ]:
@dataclass
class Order:
    order_id: int
    positions: list[Position] = None  # type: ignore
    cost: Union[int, float] = field(init=False)
    have_promo: InitVar[bool] = False

    def __post_init__(self, have_promo) -> None:   # type: ignore
        self.cost = 0
        if self.positions and len(self.positions):
            for pos in self.positions:
                self.cost += pos.cost
            if have_promo:
                self.cost = int(self.cost * (1.0 - 0.01 * DISCOUNT_PERCENTS))
            self.cost = int(self.cost)
        else:
            self.positions = []

### 1. orders  → использование map вместо компрехеншена (6)

In [ ]:
@dataclass
class Order:
    order_id: int
    positions: list[Position] = field(default_factory=list)
    cost: int = field(init=False)
    have_promo: InitVar[bool] = field(default=False)

    def __post_init__(self, have_promo: bool) -> None:
        pre_cost = sum(map(lambda item: item.cost, self.positions), start=0.0)
        self.cost = int(pre_cost * (100 - have_promo * DISCOUNT_PERCENTS) / 100)

### 2. life_game  → ethalon (1)

In [ ]:
class LifeGame:
    """
    Class for Game life
    """
    EMPTY = 0
    STONE = 1
    FISH = 2
    SHRIMP = 3

    def __init__(self, start_board: list[list[int]]) -> None:
        """
        :param start_board: rectangle-shape list of lists, which represents ocean, with numbers in cells:
                0 - empty, 1 - stone, 2 - fish, 3 - shrimp
        """
        self._board = start_board
        self._rows = len(start_board)
        self._columns = len(start_board[0])

    def __str__(self) -> str:
        lines = [' '.join([str(i) for i in line]) for line in self._board]
        return '\n'.join(lines)

    def _get_neighbour_indexes(self, i: int, j: int) -> list[tuple[int, int]]:
        return [
            (ni, nj)
            for ni in range(i - 1, i + 2)
            for nj in range(j - 1, j + 2)
            if 0 <= nj < self._columns and 0 <= ni < self._rows and (ni, nj) != (i, j)
        ]


In [ ]:
# ...
    def _get_neighbours(self, i: int, j: int, content: int) -> int:
        return sum(
            self._board[k][m] == content
            for k, m in self._get_neighbour_indexes(i, j)
        )

    def _get_next_state(self, i: int, j: int) -> int:
        current_content = self._board[i][j]
        if current_content == self.EMPTY:
            if self._get_neighbours(i, j, self.FISH) == 3:
                return self.FISH
            elif self._get_neighbours(i, j, self.SHRIMP) == 3:
                return self.SHRIMP
        elif current_content in (self.FISH, self.SHRIMP):
            if self._get_neighbours(i, j, current_content) not in (2, 3):
                return self.EMPTY
        return current_content

    def get_next_generation(self) -> list[list[int]]:
        """
        :return: updated state of the ocean board
        """
        self._board = [
            [self._get_next_state(i, j) for j in range(self._columns)] for i in range(self._rows)
        ]
        return self._board


### 2. life_game  → private instead of protected (2)

In [ ]:

    def __get_neighbors(self, i: int, j: int) -> tp.Tuple[int, int]:

        fish = 0
        shrimp = 0

        for k in range(8):
            new_i = i + self.__di[k]
            new_j = j + self.__dj[k]
            if not (0 <= new_i < self.__n and 0 <= new_j < self.__m):
                continue
            fish += self.__board[new_i][new_j] == FISH
            shrimp += self.__board[new_i][new_j] == SHRIMP

        return fish, shrimp

### 2. life_game  → дублирование кода (3)

In [ ]:
    def __fish_die(self, i: int, j: int) -> int:
        fish, shrimp = self.__get_neighbors(i, j)

        if fish <= 1 or fish >= 4:
            return EMPTY
        return FISH

    def __shrimp_die(self, i: int, j: int) -> int:
        fish, shrimp = self.__get_neighbors(i, j)

        if shrimp <= 1 or shrimp >= 4:
            return EMPTY
        return SHRIMP

### 2. life_game  → просто оставлю это здесь (4)

In [ ]:
    def _get_neighbours(self, i: int, j: int) -> list[int]:
        if self._n * self._m == 0:
            return []
        if self._n * self._m == 1:
            return []
        if self._n == 1:
            if j == 0:
                return [self._grid[0][j + 1]]
            if j == self._m - 1:
                return [self._grid[0][j - 1]]
            return [self._grid[0][j - 1], self._grid[0][j + 1]]

        if self._m == 1:
            if i == 0:
                return [self._grid[i + 1][0]]
            if i == self._n - 1:
                return [self._grid[i - 1][0]]
            return [self._grid[i - 1][0], self._grid[i + 1][0]]

        if i * j != 0 and i != self._n - 1 and j != self._m - 1:
            return self._grid[i - 1][j - 1:j + 2] + self._grid[i + 1][j - 1:j + 2] + \
                   [self._grid[i][j - 1], self._grid[i][j + 1]]
        if i == 0 and j == 0:
            return [self._grid[i + 1][j], self._grid[i][j + 1], self._grid[i + 1][j + 1]]
        if i == 0 and j == self._m - 1:
            return [self._grid[i + 1][j], self._grid[i][j - 1], self._grid[i + 1][j - 1]]
        if i == self._n - 1 and j == 0:
            return [self._grid[i - 1][j], self._grid[i][j + 1], self._grid[i - 1][j + 1]]
        if i == self._n - 1 and j == self._m - 1:
            return [self._grid[i - 1][j], self._grid[i][j - 1], self._grid[i - 1][j - 1]]
        if i == 0:
            return [self._grid[i][j - 1], self._grid[i][j + 1], self._grid[i + 1][j - 1],
                    self._grid[i + 1][j + 1], self._grid[i + 1][j]]
        if i == self._n - 1:
            return [self._grid[i][j - 1], self._grid[i][j + 1], self._grid[i - 1][j - 1],
                    self._grid[i - 1][j + 1], self._grid[i - 1][j]]
        if j == 0:
            return [self._grid[i - 1][j], self._grid[i + 1][j], self._grid[i - 1][j + 1],
                    self._grid[i + 1][j + 1], self._grid[i][j + 1]]
        if j == self._m - 1:
            return [self._grid[i - 1][j], self._grid[i + 1][j], self._grid[i - 1][j - 1],
                    self._grid[i + 1][j - 1], self._grid[i][j - 1]]
        return []

### 2. life_game  → использование filter (5)

In [ ]:

    def _get_neis(self, i: int, j: int) -> list[int]:
        positions = [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1),
                     (i + 1, j + 1), (i - 1, j - 1), (i + 1, j - 1), (i - 1, j + 1)]
        filtered_positions = filter(
            lambda x: x[0] >= 0 and x[0] < len(self.matrix)
            and x[1] >= 0 and x[1] < len(self.matrix[0]),
            positions)
        return [self.matrix[pos[0]][pos[1]] for pos in filtered_positions]

### 3. list_twist  → ethalon (1)

In [ ]:
from collections import UserList
import typing as tp


# https://github.com/python/mypy/issues/5264#issuecomment-399407428
if tp.TYPE_CHECKING:
    BaseList = UserList[tp.Optional[tp.Any]]
else:
    BaseList = UserList


class ListTwist(BaseList):
    REVERSED = ['reversed', 'R']
    FIRST = ['first', 'F']
    LAST = ['last', 'L']
    SIZE = ['size', 'S']

    def __getattr__(self, name: str) -> tp.Any:
        if name in self.REVERSED:
            return list(reversed(self.data))
        elif name in self.FIRST:
            return self.data[0]
        elif name in self.LAST:
            return self.data[-1]
        elif name in self.SIZE:
            return len(self)
        else:
            return super().__getattribute__(name)

    def __setattr__(self, name: str, value: tp.Any) -> None:
        if name in self.FIRST:
            self.data[0] = value
        elif name in self.LAST:
            self.data[-1] = value
        elif name in self.SIZE:
            if value < len(self):
                del self.data[value:]
            elif value > len(self):
                self.data += [None] * (value - len(self.data))
        else:
            return super().__setattr__(name, value)


### 3. list_twist  → repeated property (2)

In [ ]:
    # First
    @property
    def F(self) -> tp.Any:
        return self.data[0]

    @F.setter
    def F(self, new_val: tp.Any) -> None:
        self.data[0] = new_val

    @property
    def first(self) -> tp.Any:
        return self.data[0]

    @first.setter
    def first(self, new_val: tp.Any) -> None:
        self.data[0] = new_val

### 3. list_twist  → assigned property (3)

In [ ]:

    @property
    def first(self) -> tp.Any:
        return self[0]

    @first.setter
    def first(self, value: int) -> None:
        self[0] = value

    F: tp.Any = first

    @property
    def last(self) -> tp.Any:
        return self[-1]

    @last.setter
    def last(self, value: int) -> None:
        self[-1] = value

    L: tp.Any = last

### 3. list_twist  → typings ignored (4)

In [ ]:

    def __setattr__(self, attr, value):  # type: ignore
        if attr in ["reversed", "R"]:
            BaseList.data = reversed(value)
        elif attr in ["size", "S"]:
            if len(UserList.data) > value:
                BaseList.data = BaseList.data[:value]
            else:
                BaseList.data.extend([None] * (value - len(BaseList.data)))
        elif attr in ["first", "F"]:
            BaseList.data[0] = value
        elif attr in ["last", "L"]:
            BaseList.data[-1] = value